In [16]:
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import os

In [ ]:
# data
# politics : 0
# science : 1
# sports : 2
DATA = [ # sorry, I can't show you the data for copyright or something...
    

]

In [3]:
# split (labels and news)
data_X = []
data_y = []
for data in DATA:
    data_X.append(data[0]) 
    data_y.append(data[1])

In [4]:
c_0 = 0
c_1 = 0
c_2 = 0
for d in data_y:
    if d == 0:
        c_0 += 1
    elif d == 1:
        c_1 += 1
    else:
        c_2 += 1
        
print('politics : ' ,c_0)
print('science : ' ,c_1)
print('sports : ' ,c_2)
print('total : ' ,c_0 + c_1 + c_2)

politics :  1000
science :  1000
sports :  1000
total :  3000


In [5]:
def flatten(data):
    for item in data:
        if hasattr(item, '__iter__'):
            for element in flatten(item):
                yield element
        else:
            yield item

flatten(data_y)
data_y = np.array(data_y)
data_y = np_utils.to_categorical(data_y)
print('data_y_shape : ', data_y.shape)


data_y_shape :  (3000, 3)


In [6]:
#sentence to words
import re
import collections

def sentence2words(sentence):
    stopwords =["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
                "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", 
                "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", 
                "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", 
                "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
                "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
                "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", 
                "too", "very",  "can", "will", "just",  "should", "now", "s", "t", "don", "didn", "aren", "isn", "can", "re", "ll", "ve"]
    sentence = sentence.lower() # 小文字化
    sentence = sentence.replace("\n", "") # 改行削除
    sentence = re.sub(re.compile(r"[!-\?()' ‘’.,;/:-@[-`{-~]"), " ", sentence) # 記号をスペースに置き換え
    sentence = sentence.split(" ") # スペースで区切る
    sentence_words = []
    for word in sentence:
        if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None): # 数字が含まれるものは除外
            continue
        if word in stopwords: # ストップワードに含まれるものは除外
            continue
        sentence_words.append(word)        
    return sentence_words
 
# word to index
word_to_index = {}
index_to_word = {}
num_recs = 0
maxlen = 0
word_freqs = collections.Counter()
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    maxlen =max(maxlen, len(sentence_words))
    for word in sentence_words:
        word_freqs[word] += 1
    num_recs += 1
    
max_features = 4000
vocab_size = min(max_features, len(word_freqs)) + 2
word_to_index = {x[0]: i+2 for i, x in
              enumerate(word_freqs.most_common(max_features))}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1
index_to_word = {v: k for k, v in word_to_index.items()}
  

    

In [7]:
# number of headlines
print('num_recs : ', num_recs)

num_recs :  3000


In [8]:
# number of words in one headline
print('maxlen : ', maxlen)

maxlen :  25


In [9]:
print('word_freqs : ' ,len(word_freqs))

word_freqs :  6832


In [10]:
# 0 = padding
# 1 = unknown
# 2 = stop words
print('word_to_index : ', len(word_to_index))

word_to_index :  4002


In [11]:
# headline to index
data_X_vec = []
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    sentence_ids = []
    for word in sentence_words:
        if word not in word_to_index:
            sentence_ids.append(1)
            continue
        sentence_ids.append(word_to_index[word])
    data_X_vec.append(sentence_ids)
   
 

In [12]:
data_X = sequence.pad_sequences(data_X_vec, maxlen=20)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=0)

In [14]:
print('X_train : ', X_train.shape)
print('X_test : ', X_test.shape)
print('y_train : ', y_train.shape)
print('y_test : ', y_test.shape)

X_train :  (2400, 20)
X_test :  (600, 20)
y_train :  (2400, 3)
y_test :  (600, 3)


In [41]:
embeddings_index = {}
GLOVE_DIR = os.path.join('Downloads', 'glove.6B')
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), "r", encoding="utf-8" )
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [42]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_to_index) , EMBEDDING_DIM))
for word, i in word_to_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [43]:
embedding_matrix.shape

(4003, 300)

In [58]:
# Build model
model = Sequential()
model.add(Embedding(len(word_to_index) ,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=20))
model.add(Dropout(0.3))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 20, 300)           1200900   
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 300)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 195       
_________________________________________________________________
activation_12 (Activation)   (None, 3)                 0         
Total params: 1,294,535
Trainable params: 1,294,535
Non-trainable params: 0
_________________________________________________________________


In [59]:
history = model.fit(X_train, y_train, batch_size=20,
                    epochs=2,
                    validation_data=(X_test, y_test))

Train on 2400 samples, validate on 600 samples
Epoch 1/2
2400/2400 [==============================] - 10s 4ms/step - loss: 0.6011 - acc: 0.7633 - val_loss: 0.2866 - val_acc: 0.9100
Epoch 2/2
2400/2400 [==============================] - 8s 3ms/step - loss: 0.3088 - acc: 0.8892 - val_loss: 0.2497 - val_acc: 0.9217


In [60]:
score, acc = model.evaluate(X_test, y_test, batch_size=10)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score, acc))


        
for i in range(10):
    idx = np.random.randint(len(X_test))
    xtest = X_test[idx].reshape(1,20)
    
    ypred = model.predict(xtest)
    predicted_class_indices=np.argmax(ypred,axis=1)
    ylabel = y_test[idx]
    ylabel = np.argmax(ylabel)
    headline = " ".join([index_to_word[x] for x in xtest[0].tolist() if x != 0])
    print('predict :', predicted_class_indices, 'label :', ylabel,'headline :', headline)
   

600/600 [==============================] - 0s 630us/step
Test score: 0.250, accuracy: 0.922
predict : [2] label : 2 headline : sloane stephens art defending
predict : [0] label : 0 headline :       democrats expect female front runner  three 
predict : [2] label : 2 headline : brian UNK UNK real victims n c  UNK
predict : [2] label : 2 headline : u  open draw sets possible early meeting williams sisters
predict : [0] label : 0 headline : trump women  another writer looks answers
predict : [0] label : 0 headline : abroad america  almost
predict : [0] label : 0 headline : former top f b  lawyer says rosenstein serious taping trump
predict : [0] label : 0 headline : great ad  compelling characters  mass audience democratic candidates 
predict : [2] label : 2 headline : u c f  best college football team florida   respect 
predict : [0] label : 0 headline : republicans rushing save house seats onslaught democratic money
